Este cuaderno contiene el código en el que se ha tenido que modificar la función para realizar el último apartado, incluyendo algunos cálculos dentro de la función main, ya que, dependían del número de patrones.

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import math
from numpy import random as randomp
from random import seed
from random import random
from datetime import datetime
from numba import jit


#semilla para num aleatorios
now = datetime.now()
seed(now.microsecond)
#randomp.seed(now.microsecond)



#paso 0 (inicializar) globales
sp=randomp.choice([0,1],size=(npart,npart)) 
npart=20
paso=npart*npart



#patron
patron1=np.loadtxt("patron1.txt")
uno=np.loadtxt("unouno.txt")
dos=np.loadtxt("dosdos.txt")
tres=np.loadtxt("tres.txt")
cuatro=np.loadtxt("cuatro.txt")
#patron1=randomp.choice([0,1],size=(npart,npart))









######### calculo del hamiltoniano #######
@jit(nopython=True)
def hamilton1(s,theta,omeg):
    suma1=0
    suma2=0
    for i in range (npart):
        for j in range (npart):
            suma2=suma2+theta[i][j]*s[i][j]
            for k in range(npart):
                for l1 in range(npart):
                    suma1=suma1+omeg[i,j,k,l1]*s[i][j]*s[k][l1]

    return -0.5*suma1+suma2
#########################################
@jit(nopython=True)
def hamilton(s,a,b,theta,omeg):
    suma=0
    for i in range(npart):
        if(i!=a):
            for j in range(npart):
                if(j!=b):
                    suma=suma+omeg[i][j][a][b]*s[i][j]
                
    return (1-2*s[a][b])*(theta[a][b]-0.5*omeg[a][b][a][b]-suma)


########### calculo del solapamiento
@jit(nopython=False)
def solapamiento(s,pos,zeta,a):
    suma=0
    for i in range(npart):
        for j in range(npart):
            suma=suma+(zeta[pos][i][j]-a[pos])*(s[i][j]-a[pos])
    
    return suma/(npart*npart*a[pos]*(1-a[pos]))


############################
def main(t,montec,p):

##definiciones previas que antes estaban fuera de la función
    
    zeta=randomp.choice([0,1],size=(p,npart,npart))
    ##################suma de a
    a=np.zeros(p)
    for l in range(p):
        for i in range (npart):
            for j in range (npart):
                a[l]=a[l]+zeta[l][i][j]      
    a=a/(npart*npart)
            

    theta=np.zeros((npart,npart))
                    
                    

    def omega(i,j,k,l):
        suma=0
        if ((i!=k) or (j!=l)):
            for n2 in range (p):
                suma=suma+(zeta[n2][i][j]-a[n2])*(zeta[n2][k][l]-a[n2])
        return suma/(npart*npart)
    omeg=np.zeros((npart,npart,npart,npart))
    for i in range(npart):
        for j in range(npart):
            for k in range(npart):
                for l in range(npart):
                    omeg[i][j][k][l]=omega(i,j,k,l)
    #umbral de disparo:(calculamos una vez todos los umbrales de disparo para no tener que calcularlos en cada iteración,
    # ya que es fijo a partir de los patrones) 
    for i in range(npart):
        for j in range(npart):
            for k in range(npart):
                for l in range(npart):
                    theta[i][j]=theta[i][j]+omeg[i,j,k,l]
    theta=0.5*theta                
    ###################




#variables de la función
    itera=montec*paso #numero iteraciones

    s=np.copy(sp)

    

    for i in range(itera+1):
        n = randomp.randint(npart)
        m = randomp.randint(npart)
        

        #generamos un sitema exactamente igual salvo que le damos la vuelta al spin que hemos generado aleatoriamente
        #sp=np.copy(s)
        #if(s[n][m]==0):
        #    sp[n][m]=1
        #else:
        #    sp[n][m]=0

        

        #minimo=min(1,math.exp(-(hamilton(sp)-hamilton(s))/t))
        minimo=min(1,np.exp(-hamilton(s,n,m,theta,omeg)/t))

      
      

        xi=random()
        #if(xi<minimo):
            #s[n][m]=sp[n][m]

        if(xi<minimo):
            if(s[n][m]==0):
                s[n][m]=1
            else:
                s[n][m]=0
          
    vecsolap=np.zeros(p)
    contador=0
    vecsolap=np.zeros(p)
    for i in range(p):
        vecsolap[i]=abs(solapamiento(s,i,zeta,a))
    
    
    maximo=vecsolap.max()

    return maximo

#%prun main()